In [1]:
import torch

In [2]:
# These are the inputs to the attention mechanism, which represents the respectve vectors of each word in the sequence.
inputs = torch.tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]])

In [3]:
x_2 = inputs[1]
dim_in = inputs.shape[1]
dim_out = 2

In [4]:
torch.manual_seed(123)
w_query = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
w_key = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
w_value = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
print(w_query, w_key, w_value)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]]) Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]]) Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [5]:
query_of_x_2 = x_2 @ w_query
key_of_x_2 = x_2 @ w_key
value_of_x_2 = x_2 @ w_value
print(query_of_x_2, key_of_x_2, value_of_x_2)

tensor([0.7659, 2.6901]) tensor([0.8265, 2.1365]) tensor([0.7239, 1.8921])


In [6]:
querys = inputs @ w_query
keys = inputs @ w_key
values = inputs @ w_value
print(querys, keys, values)
print(querys.shape, keys.shape, values.shape)

tensor([[ 0.0427, -0.0946],
        [ 0.7659,  2.6901],
        [ 0.3152,  0.3812],
        [ 0.0394, -0.5367],
        [-0.3080, -0.9217],
        [-1.1428, -3.2289]]) tensor([[-0.0399, -0.2107],
        [ 0.8265,  2.1365],
        [ 0.0866, -0.1257],
        [-0.2389, -0.1278],
        [-0.2975, -0.3162],
        [-0.9767, -1.8298]]) tensor([[-0.0508, -0.1449],
        [ 0.7239,  1.8921],
        [ 0.0138,  0.0371],
        [ 0.1393, -0.6406],
        [-0.0595, -0.6196],
        [-0.6303, -2.0401]])
torch.Size([6, 2]) torch.Size([6, 2]) torch.Size([6, 2])


In [7]:
atten_of_x_2 = query_of_x_2.dot(key_of_x_2)
print(atten_of_x_2)

tensor(6.3803)


In [8]:
all_atten = querys @ keys.T
print(all_atten)

tensor([[ 1.8226e-02, -1.6681e-01,  1.5579e-02,  1.8998e-03,  1.7218e-02,
          1.3139e-01],
        [-5.9739e-01,  6.3803e+00, -2.7178e-01, -5.2682e-01, -1.0784e+00,
         -5.6703e+00],
        [-9.2905e-02,  1.0749e+00, -2.0629e-02, -1.2401e-01, -2.1429e-01,
         -1.0054e+00],
        [ 1.1151e-01, -1.1140e+00,  7.0863e-02,  5.9182e-02,  1.5798e-01,
          9.4350e-01],
        [ 2.0651e-01, -2.2238e+00,  8.9181e-02,  1.9139e-01,  3.8307e-01,
          1.9873e+00],
        [ 7.2598e-01, -7.8431e+00,  3.0688e-01,  6.8573e-01,  1.3610e+00,
          7.0244e+00]])


In [22]:
atten_weights = torch.softmax(all_atten, dim = 1)                                            
print(atten_weights)
print(atten_weights[2].sum())

tensor([[1.6860e-01, 1.4012e-01, 1.6816e-01, 1.6587e-01, 1.6843e-01, 1.8881e-01],
        [9.2893e-04, 9.9621e-01, 1.2864e-03, 9.9685e-04, 5.7419e-04, 5.8191e-06],
        [1.3251e-01, 4.2602e-01, 1.4244e-01, 1.2845e-01, 1.1736e-01, 5.3208e-02],
        [1.5271e-01, 4.4836e-02, 1.4663e-01, 1.4493e-01, 1.5998e-01, 3.5092e-01],
        [9.9106e-02, 8.7224e-03, 8.8134e-02, 9.7618e-02, 1.1824e-01, 5.8818e-01],
        [1.8241e-03, 3.4638e-07, 1.1996e-03, 1.7521e-03, 3.4420e-03, 9.9178e-01]])
tensor(1.)


In [23]:
context_of_x_2 = atten_weights[1] @ values
print(context_of_x_2)

tensor([0.7212, 1.8838])


In [27]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.w_query = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
        self.w_key = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
        self.w_value = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad = False)
    def forward(self, x):
        querys = x @ self.w_query
        keys = x @ self.w_key
        values = x @ self.w_value
        all_atten = querys @ keys.T
        atten_weights = torch.softmax(all_atten, dim = 1)
        context_vecs = atten_weights @ values
        return context_vecs

In [28]:
torch.manual_seed(123)
self_attention = SelfAttention_v1(dim_in, dim_out)
print(self_attention(inputs))

tensor([[-0.0107, -0.3489],
        [ 0.7212,  1.8838],
        [ 0.2810,  0.5286],
        [-0.1838, -0.8397],
        [-0.3617, -1.3303],
        [-0.6251, -2.0268]])


In [30]:
import torch.nn as nn
class SelfAttention_v2(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.w_query = nn.Linear(dim_in, dim_out, bias = False)
        self.w_key = nn.Linear(dim_in, dim_out, bias = False)
        self.w_value = nn.Linear(dim_in, dim_out, bias = False)
    
    def forward(self, x):
        querys = self.w_query(x)
        keys = self.w_key(x)
        values = self.w_value(x)

        all_atten = querys @ keys.T
        atten_weights = torch.softmax(all_atten, dim = 1)
        context_vecs = atten_weights @ values
        return context_vecs

In [31]:
self_attention_v2 = SelfAttention_v2(dim_in, dim_out)
print(self_attention_v2(inputs))

tensor([[-0.1359,  0.0974],
        [-0.1075,  0.2320],
        [-0.0438,  0.1442],
        [-0.2889, -0.0032],
        [-0.3373, -0.0185],
        [-0.6094, -0.1934]], grad_fn=<MmBackward0>)
